<a href="https://colab.research.google.com/github/MPGarg/S4/blob/main/EVA8_Session_3_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=0) 
        self.bn1 = nn.BatchNorm2d(16)
        self.drop1 = nn.Dropout(0.05)
        
        self.conv2 = nn.Conv2d(16, 32, 3, padding=0)
        self.bn2 = nn.BatchNorm2d(32)
        self.drop2 = nn.Dropout(0.05)
        
        self.conv3 = nn.Conv2d(32, 8, 1, padding=0)
        self.pool3 = nn.MaxPool2d(2, 2)

        self.conv4 = nn.Conv2d(8, 16, 3, padding=0)
        self.bn4 = nn.BatchNorm2d(16)
        self.drop4 = nn.Dropout(0.05)
        
        self.conv5 = nn.Conv2d(16, 32, 3, padding=0)
        self.bn5 = nn.BatchNorm2d(32)
        self.drop5 = nn.Dropout(0.05)
        
        self.conv6 = nn.Conv2d(32, 16, 1, padding=0)
        self.pool6 = nn.MaxPool2d(2, 2)

        self.conv7 = nn.Conv2d(16, 32, 3, padding=0)
        
        self.pool8 = nn.AvgPool2d(2,2)
        
        self.fc1 = nn.Linear(32, 10)
        
    def forward(self, x):
        x = self.drop1(self.bn1(F.relu(self.conv1(x))))
        x = self.drop2(self.bn2(F.relu(self.conv2(x))))
        x = self.pool3(self.conv3(x))
        x = self.drop4(self.bn4(F.relu(self.conv4(x))))
        x = self.drop5(self.bn5(F.relu(self.conv5(x))))
        x = self.pool6(self.conv6(x))
        x = F.relu(self.conv7(x))
        x = self.pool8(x)
        
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        return F.log_softmax(x)

In [3]:
#!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
       BatchNorm2d-2           [-1, 16, 26, 26]              32
           Dropout-3           [-1, 16, 26, 26]               0
            Conv2d-4           [-1, 32, 24, 24]           4,640
       BatchNorm2d-5           [-1, 32, 24, 24]              64
           Dropout-6           [-1, 32, 24, 24]               0
            Conv2d-7            [-1, 8, 24, 24]             264
         MaxPool2d-8            [-1, 8, 12, 12]               0
            Conv2d-9           [-1, 16, 10, 10]           1,168
      BatchNorm2d-10           [-1, 16, 10, 10]              32
          Dropout-11           [-1, 16, 10, 10]               0
           Conv2d-12             [-1, 32, 8, 8]           4,640
      BatchNorm2d-13             [-1, 32, 8, 8]              64
          Dropout-14             [-1, 3

<ipython-input-2-108c2dff5a76>:44: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [4]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [5]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'epoch={epoch} loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.1f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-2-108c2dff5a76>:44: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
epoch=1 loss=0.05318264663219452 batch_id=468: 100%|██████████| 469/469 [00:15<00:00, 29.51it/s]



Test set: Average loss: 0.0573, Accuracy: 9833/10000 (98.3%)



epoch=2 loss=0.02729177661240101 batch_id=468: 100%|██████████| 469/469 [00:15<00:00, 31.16it/s]



Test set: Average loss: 0.0548, Accuracy: 9799/10000 (98.0%)



epoch=3 loss=0.051630523055791855 batch_id=468: 100%|██████████| 469/469 [00:15<00:00, 29.95it/s]


...


Test set: Average loss: 0.0201, Accuracy: 9939/10000 (99.4%)



epoch=18 loss=0.0028530091512948275 batch_id=468: 100%|██████████| 469/469 [00:15<00:00, 30.97it/s]



Test set: Average loss: 0.0200, Accuracy: 9946/10000 (99.5%)



epoch=19 loss=0.000873862998560071 batch_id=468: 100%|██████████| 469/469 [00:15<00:00, 31.01it/s]



Test set: Average loss: 0.0208, Accuracy: 9935/10000 (99.3%)

